# 数据读取与处理

In [1]:
import pandas as pd

video_list = pd.read_excel('./data/video_list.xlsx')
speech_text = pd.read_excel('./data/speech_text.xlsx')


In [2]:
video_list['AwemeId'] = video_list['AwemeId'].astype(str)
video_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   品牌                    73 non-null     object
 1   AwemeId               73 non-null     object
 2   AwemeDesc             73 non-null     object
 3   AwemePubTime          73 non-null     object
 4   BloggerName           73 non-null     object
 5   BloggerPlatform_Fans  73 non-null     object
 6   LikeCount             73 non-null     object
 7   CommentCount          73 non-null     object
 8   ShareCount            73 non-null     object
dtypes: object(9)
memory usage: 5.3+ KB


In [3]:
speech_text['VideoId'] = speech_text['VideoId'].astype(str)
speech_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   视频标题     73 non-null     object
 1   视频时长     73 non-null     int64 
 2   VideoId  73 non-null     object
 3   视频文案     73 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.4+ KB


# 两表数据连接

In [4]:
merge_data = pd.merge(video_list, speech_text, how='inner', left_on='AwemeId', right_on='VideoId')

merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 72
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   品牌                    73 non-null     object
 1   AwemeId               73 non-null     object
 2   AwemeDesc             73 non-null     object
 3   AwemePubTime          73 non-null     object
 4   BloggerName           73 non-null     object
 5   BloggerPlatform_Fans  73 non-null     object
 6   LikeCount             73 non-null     object
 7   CommentCount          73 non-null     object
 8   ShareCount            73 non-null     object
 9   视频标题                  73 non-null     object
 10  视频时长                  73 non-null     int64 
 11  VideoId               73 non-null     object
 12  视频文案                  73 non-null     object
dtypes: int64(1), object(12)
memory usage: 8.0+ KB


# 提取与处理

In [18]:
from docx import Document
import docx
  

def seconds_to_time(seconds): 
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return "%02d小时%02d分钟%02d秒" % (h, m, s)


document = Document()
pinpai_list = []
title_list = []
for i in range(merge_data.shape[0]):
    pinpai = merge_data.iloc[i]['品牌']
    title = merge_data.iloc[i]['视频标题'].split('_')[0]
    BloggerName = merge_data.iloc[i]['BloggerName']
    video_address = 'douyin.com/video/' + merge_data.iloc[i]['AwemeId']
    video_content = merge_data.iloc[i]['视频文案']
    video_time_seconds = merge_data.iloc[i]['视频时长']
    video_time = seconds_to_time(video_time_seconds)
    # 品牌
    if pinpai not in pinpai_list:
        pinpai_list.append(pinpai)
        document.add_heading(pinpai, level=1)

    # 视频标题
    if title not in title_list:
        title_list.append(title)
        document.add_heading(title, level=2)

    # 其余项
    document.add_paragraph(f'达人名称：{BloggerName}')
    # 超链接视频地址
    v_a = document.add_paragraph('视频地址：')
    run = v_a.add_run(f'{video_address}')
    hyperlink = docx.oxml.shared.OxmlElement("w:hyperlink")
    hyperlink.set(docx.oxml.ns.qn("w:anchor"), '微软雅黑')
    run._r.append(hyperlink)

    document.add_paragraph(f'视频时长：{video_time}')
    document.add_paragraph(video_content)

document.save('demo.docx')
# print(pinpai_list)
